# M04 Thống kê

## Mục đích

Giới thiệu thư viện Scipy về các hàm toán học và Statsmodels về các mô hình thống kê. Bài này sẽ tập trung giới thiệu thư viện con của Scipy dành cho thống kê.


## Phân bố xác suất

Mặc dù NumPy cho phép chúng ta tạo số ngẫu nhiên theo một phân bố xác suất nào đó, nó không thể thực hiện được việc tính toán các hàm xác suất (ví dụ, tính $P(X < x)$ với $X \sim N(\mu, \sigma^2)$). Bạn có thể sử dụng thư viện con `scipy.stats` cho việc này.

Chúng ta sẽ xem thử ví dụ tính $P(X < 1.65); X \sim N(0, 1)$.

In [1]:
import scipy.stats as stats

stats.norm.cdf(1.65)

0.9505285319663519

Trong trường hợp phân bố của bạn có các tham số khác với tham số mặc định (ví dụ, phân bố chuẩn với trung bình $\mu$ và phương sai $\sigma^2$), bạn có thể cung cấp các thông tin này cho Scipy. Ví dụ sau đấy tính $P(X < 8); X \sim N(2, 3)$

In [2]:
stats.norm(2, 3).cdf(8)

0.9772498680518208

Nếu bạn chưa quen với các khái niệm về hàm xác suất thì bạn sẽ cần đi học một khóa về lí thuyết xác suất. Mình sẽ chỉ tóm tắt các hàm mà Scipy cung cấp cho mỗi phân bố xác suất.

Hàm xác suất | Ví dụ
-------------|------------------------------------
CDF          | `stats.norm.cdf(x)`
PDF (liên tục) | `stats.norm.pdf(x)`
PMF (rời rạc) | `stats.poisson.pmf(x)`
Hàm ngược của CDF | `stats.norm.ppf(x)`

In [3]:
print("X ~ Pois(mu=5), P(X=4) =" , stats.poisson(mu=5).pmf(4))
print("X ~ N(0, 1), P(X < a) = 0.95, a =", stats.norm.ppf(0.95))

X ~ Pois(mu=5), P(X=4) = 0.17546736976785063
X ~ N(0, 1), P(X < a) = 0.95, a = 1.6448536269514722


Các hàm này cũng nhận đối số `x` là một danh sách hoặc mảng.

In [4]:
xs = list(range(5))
ys = stats.binom(n=10, p=0.4).pmf(xs)

print("X ~ Binomial(10, 0.4)")
for x, y in zip(xs, ys):
    print(f"P(X = {x}) = {y:.3f}")

X ~ Binomial(10, 0.4)
P(X = 0) = 0.006
P(X = 1) = 0.040
P(X = 2) = 0.121
P(X = 3) = 0.215
P(X = 4) = 0.251


Bạn có thể sử dụng các tính năng này của Scipy trong việc tính toán thống kê, chẳng hạn, tính p-value của kiểm định *t*-test một biến:

$$\text{p} = P(\mu \le \bar{x} | H_0)
= P\left(\frac{\mu - \mu_0}{s_{\bar{X}}} \le \frac{m - \mu_0}{s_{\bar{X}}}\right)
= P(t \le t_m) = t_{n-1}(t_m)$$
$$s_{\bar{X}} = \frac{s_X}{\sqrt{n}};
t_m = \frac{m - \mu_0}{s_{\bar{X}}}$$

Ví dụ, nghiên cứu của bạn làm trên 100 người tính ra được chỉ số MCV trung bình là 85 fL (SD 25) và bạn muốn so sánh với trung bình quần thể là 90 fL.

In [5]:
from math import sqrt

n, m, s, m0 = 100, 85, 25, 90
t_m = (m - m0) / s * sqrt(n)
print("p value =", stats.t(n-1).cdf(t_m))

p value = 0.024119846686316487


Nếu không sử dụng các tính toán phức tạp, bạn có thể sử dụng sẵn các tính năng mà Scipy cung cấp. Chẳng hạn như hàm tính t-test một biến.

In [6]:
a = stats.norm(loc=m, scale=s).rvs(size=n, random_state=0)    # Tương tự np.random.norm()
print("mean = {}, SD = {}".format(a.mean(), a.std()))

t, p = stats.ttest_1samp(a, m0, alternative="less")
print(f"t = {t:.3f}, p value = {p:.3f}")

mean = 86.49520038836212, SD = 25.197056117914492
t = -1.384, p value = 0.085


## Mô hình thống kê

Thư viện Statsmodels cung cấp các mô hình thống kê, chủ yếu là hồi quy. Việc chạy các mô hình hồi quy trên Statsmodels khá giống với trên R: (1) lập mô hình, (2) fit số liệu, và (3) trình bày kết quả / dự đoán / v.v. Nêu không sử dụng các tính năng chuyên sâu, thư viện con `statsmodels.api` là đủ cho bạn.

In [7]:
import numpy as np
import statsmodels.api as sm

np.random.seed(0)

# Giả lập số liệu
n = 200
sex = np.random.choice([0, 1], n, replace=True, p=[0.3, 0.7])
height = np.round(np.where(sex == 0, np.random.normal(45, 10, n), np.random.normal(50, 12, n)))
bloodtest = np.round(np.random.gamma(7, 2, n), 2)
qolscore = np.random.poisson(6, n)
xs = np.array([sex, height, bloodtest, qolscore]).T
y = 1.5 + sex * 0.3 + height * 0.01 + bloodtest * 3.2 + qolscore * 1.16 + np.random.normal(0, 1, n)

# Hồi quy tuyến tính đa biến
model = sm.OLS(y, sm.add_constant(xs))    # Lập mô hình
result = model.fit()                      # Fit số liệu
result.summary()                          # Trình bày kết quả

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 1.290e+04
Date:                Sun, 14 Aug 2022   Prob (F-statistic):          4.39e-235
Time:                        23:07:40   Log-Likelihood:                -281.37
No. Observations:                 200   AIC:                             572.7
Df Residuals:                     195   BIC:                             589.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8303      0.419      1.984      0.049       0.005       1.656
x1             0.2607      0.157      1.657      0.099      -0.050       0.571
x2             0.0113      0.006      1.757      0.080      -0.001       0.024
x3             3.2128      0.014    223.395      0.000       3.184       3.241
x4             1.2277      0.030     40.866      0.000       1.168       1.287
==============================================================================
Omnibus:                        0.551   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.759   Jarque-Bera (JB):                0.672
Skew:                           0.109   Prob(JB):                        0.715
Kurtosis:                       2.819   Cond. No.                         305.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Mô hình mà mình sử dụng ở trên là Ordinary Least Squares (OLS), là mô hình hồi quy tuyến tính mà bạn thường được dạy. Để sử dụng OLS, chúng ta cung cấp cho nó số liệu của biến trả lời (`endog`) và các biến giải thích (`exog`). Tuy nhiên, bạn có thể thấy mình sử dụng thêm hàm `sm.add_constant()`. Hàm này sẽ thêm một cột các số 1 vào bên trái của ma trận biến giải thích, cột này dành cho hệ số của intercept hay constant ($\beta_0$) trong mô hình hồi quy.

Nếu đã sử dụng thư viện Pandas và quen với cách viết công thức của R, bạn có thể làm một cách khác mà không cần thêm cột constant này.

In [8]:
import pandas as pd

d = pd.DataFrame({
    "sex": sex,
    "height": height,
    "bloodtest": bloodtest,
    "qolscore": qolscore,
    "y": y
})

fml = "y ~ sex + height + bloodtest + qolscore"
model = sm.OLS.from_formula(fml, data=d)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 1.290e+04
Date:                Sun, 14 Aug 2022   Prob (F-statistic):          4.39e-235
Time:                        23:07:40   Log-Likelihood:                -281.37
No. Observations:                 200   AIC:                             572.7
Df Residuals:                     195   BIC:                             589.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8303      0.419      1.984      0.049       0.005       1.656
sex            0.2607      0.157      1.657      0.099      -0.050       0.571
height         0.0113      0.006      1.757      0.080      -0.001       0.024
bloodtest      3.2128      0.014    223.395      0.000       3.184       3.241
qolscore       1.2277      0.030     40.866      0.000       1.168       1.287
==============================================================================
Omnibus:                        0.551   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.759   Jarque-Bera (JB):                0.672
Skew:                           0.109   Prob(JB):                        0.715
Kurtosis:                       2.819   Cond. No.                         305.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Chạy những mô hình phức tạp hơn như GLM cũng không khó khăn gì.

In [9]:
# Tạo dữ liệu ngẫu nhiên
lam = np.exp(-2 + 0.7 * sex)

d = pd.DataFrame({
    "sex": sex,
    "y": np.random.poisson(lam)
})

# Hồi quy Poisson
fml = "y ~ sex"
model = sm.GLM.from_formula(fml, data=d,
    family=sm.families.Poisson())
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  200
Model:                            GLM   Df Residuals:                      198
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -109.35
Date:                Sun, 14 Aug 2022   Deviance:                       139.64
Time:                        23:07:40   Pearson chi2:                     195.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.03159
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.3191      0.408     -5.681      0.000      -3.119      -1.519
sex            0.9956      0.440      2.262      0.024       0.133       1.858
==============================================================================
"""

Hệ số hồi quy và khoảng tin cậy 95% đang ở dạng logarit cơ số tự nhiên. Chúng ta có thể mũ e để thu được IRR (95%CI) của nó. Dữ liệu về hệ số hồi quy, khoảng tin cậy 95%, p-value, v.v. có kiểu dữ liệu `pandas.Series` và `pandas.DataFrame`. Chúng ta sẽ tìm hiểu kĩ về các kiểu dữ liệu này cũng như các thao tác để kết hợp các chỉ số kia vào thành một bảng trong chương về Pandas.

In [10]:
np.exp(result.params)

Intercept    0.098361
sex          2.706235
dtype: float64

In [11]:
np.exp(result.conf_int())

,0,1
Intercept,0.044190,0.218939
sex,1.142212,6.411865


---

[Bài trước](./03_slicing.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()